<a href="https://colab.research.google.com/github/GabrielAzqe/Projeto-de-LMN/blob/main/projeto_pln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 1. INSTALAÇÃO (Colab)
# ==========================================
!pip install -qU google-search-results langchain-google-genai langchain langchain-community

In [ ]:
# ==========================================
# 1. IMPORTS E CONFIGURAÇÃO
# ==========================================
import os
import getpass
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from serpapi import GoogleSearch
from datetime import datetime, timedelta
from IPython.display import Markdown, display

# ==========================================
# FUNÇÃO DE LOG PADRONIZADA
# ==========================================
def log(msg):
    print(f"\n🟨 [LOG] {msg}\n")

# ==========================================
# PRETTY PRINT – saída estruturada da API (truncada)
# ==========================================
def print_estruturado(titulo, dados, max_linhas=20):
    print(f"\n🟦 {titulo} (mostrando até {max_linhas} linhas) \n")
    try:
        json_str = json.dumps(dados, indent=4, ensure_ascii=False).split("\n")
        for linha in json_str[:max_linhas]:
            print(linha)
        if len(json_str) > max_linhas:
            print("... (saida truncada)")
    except:
        print(dados)

# ==========================================
# FUNÇÃO PARA ESCAPAR CARACTERES QUE QUEBRAM MARKDOWN
# ==========================================
def escapar_markdown(texto):
    if not texto:
        return ""
    texto = texto.replace("R$", "R\\$")  # Escapa R$
    texto = texto.replace("$", "\\$")    # Escapa outros $ isolados
    return texto

# ==========================================
# 2 - Configuração das chaves
# ==========================================
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Cole sua Google AI Key: ")

if "SERPAPI_API_KEY" not in os.environ:
    os.environ["SERPAPI_API_KEY"] = getpass.getpass("Cole sua SerpAPI Key: ")

SERPAPI_KEY = os.environ["SERPAPI_API_KEY"]

# ==========================================
# FUNÇÃO PARA CHECAR VOOS DISPONÍVEIS
# ==========================================
def verificar_voo_disponivel(data_ida, data_volta, origem_iata, destino_iata):
    try:
        params = {
            "engine": "google_flights",
            "departure_id": origem_iata,
            "arrival_id": destino_iata,
            "outbound_date": data_ida,
            "return_date": data_volta,
            "api_key": SERPAPI_KEY,
            "hl": "pt"
        }
        resultado = GoogleSearch(params).get_dict()
        voos = resultado.get("best_flights", []) or resultado.get("other_flights", [])
        return len(voos) > 0
    except:
        return False

# ==========================================
# 3. GERAR DATAS + INTERESSES VIA LLM
# ==========================================
def gerar_dados_viagem(origem, destino):
    log(f"Gerando datas e interesses para a viagem {origem} → {destino} via LLM")
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    hoje = datetime.today()
    limite_min = hoje + timedelta(days=30)
    limite_max = hoje + timedelta(days=300)

    prompt = f"""
Escolha datas ideais para uma viagem de {origem} até {destino}.
Regras:
- viagem deve ter entre 5 e 12 dias.
- escolha um mês em alta ou média temporada.
- a ida deve ser entre {limite_min.date()} e {limite_max.date()}.
- interesses devem ser variados (cultura, gastronomia, natureza etc.)

Responda no formato:
IDA: AAAA-MM-DD
VOLTA: AAAA-MM-DD
INTERESSES: lista separada por vírgula
"""

    resposta = llm.invoke(prompt).content
    linhas = resposta.split("\n")
    data_ida = linhas[0].split(":")[1].strip()
    data_volta = linhas[1].split(":")[1].strip()
    interesses = linhas[2].split(":")[1].strip()

    log(f"Datas geradas: {data_ida} → {data_volta}")
    log(f"Interesses: {interesses}")
    return data_ida, data_volta, interesses

# ==========================================
# 4. TRADUZIR CIDADE -> IATA
# ==========================================
def descobrir_aeroporto(cidade):
    log(f"Identificando código IATA de {cidade}...")
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    prompt = f"""
Qual é o código IATA (3 letras) do principal aeroporto internacional de: {cidade}?
Responda apenas a sigla.
"""
    codigo = llm.invoke(prompt).content.strip()
    log(f"Código encontrado: {codigo}")
    return codigo

# ==========================================
# 5. SERPAPI — BUSCA DE VOOS
# ==========================================
def buscar_voos_reais(origem, destino_iata, data_ida, data_volta):
    log(f"Buscando voos reais {origem} → {destino_iata}")
    try:
        params = {
            "engine": "google_flights",
            "departure_id": origem,
            "arrival_id": destino_iata,
            "outbound_date": data_ida,
            "return_date": data_volta,
            "currency": "BRL",
            "hl": "pt",
            "api_key": SERPAPI_KEY
        }
        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (VOOS)", results)
        voos = results.get("best_flights", []) or results.get("other_flights", [])
        if not voos:
            return "Nenhum voo encontrado."
        lista = []
        for v in voos[:3]:
            preco = v.get("price", "N/A")
            duracao = v["flights"][0].get("duration", "N/A")
            cia = v["flights"][0].get("airline", "Companhia")
            lista.append(f"- {cia}: R$ {preco} ({duracao} min)")
        return "\n".join(lista)
    except Exception as e:
        return f"Erro voos: {e}"

# ==========================================
# 6. SERPAPI — HOTÉIS
# ==========================================
def buscar_hoteis_reais(destino, checkin, checkout):
    log(f"Buscando hotéis reais em {destino}")
    try:
        params = {
            "engine": "google_hotels",
            "q": f"hotéis em {destino}",
            "check_in_date": checkin,
            "check_out_date": checkout,
            "currency": "BRL",
            "hl": "pt",
            "api_key": SERPAPI_KEY
        }
        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (HOTÉIS)", results)
        props = results.get("properties", [])[:3]
        if not props:
            return "Nenhum hotel encontrado."
        lista = []
        for h in props:
            nome = h.get("name", "Hotel")
            preco = h.get("rate_per_night", {}).get("lowest", "N/A")
            nota = h.get("overall_rating", "S/N")
            lista.append(f"- {nome}: {preco} (Nota: {nota})")
        return "\n".join(lista)
    except Exception as e:
        return f"Erro hotéis: {e}"

# ==========================================
# 7. SERPAPI — ATRAÇÕES
# ==========================================
def pesquisar_atracoes(destino, interesses):
    log(f"Pesquisando atrações reais em {destino} baseadas em: {interesses}")
    try:
        params = {
            "engine": "google",
            "q": f"melhores atrações {destino} {interesses}",
            "api_key": SERPAPI_KEY,
            "hl": "pt",
            "gl": "br"
        }
        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (ATRAÇÕES)", results)
        org = results.get("organic_results", [])
        if not org:
            return "Nenhuma atração encontrada."
        return "\n".join(f"- {i.get('title')}: {i.get('snippet','')}" for i in org[:4])
    except:
        return "Erro atrações."

# ==========================================
# 8. SERPAPI — CÂMBIO
# ==========================================
def buscar_cambio(pais):
    log(f"Buscando câmbio para o país: {pais}")
    try:
        params = {
            "engine": "google",
            "q": f"cotação moeda {pais} hoje",
            "api_key": SERPAPI_KEY,
            "hl": "pt",
            "gl": "br"
        }
        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (CÂMBIO)", results)
        org = results.get("organic_results", [])
        if not org:
            return "Câmbio não encontrado."
        return org[0].get("snippet", "Sem informações.")
    except Exception as e:
        return f"Erro câmbio: {e}"

# ==========================================
# 9. SERPAPI — HISTÓRIA
# ==========================================
def pesquisar_historia(destino):
    log(f"Pesquisando história de {destino}")
    try:
        params = {
            "engine": "google",
            "q": f"história de {destino}",
            "api_key": SERPAPI_KEY,
            "hl": "pt",
            "gl": "br"
        }
        results = GoogleSearch(params).get_dict()
        print_estruturado("RESULTADO SERPAPI (HISTÓRIA)", results)
        org = results.get("organic_results", [])
        if not org:
            return "História não encontrada."
        return "\n".join(f"- {item.get('title')}: {item.get('snippet','')}" for item in org[:3])
    except:
        return "Erro história."

# ==========================================
# 10. ORQUESTRADOR COMPLETO COM CHECK DE VOOS (RETORNA DADOS PARA PROMPT)
# ==========================================
def agente_de_viagens(origem_usuario, destino_usuario):
    log("Iniciando agente de viagens...")

    tentativas = 0
    max_tentativas = 3
    while tentativas < max_tentativas:
        data_ida, data_volta, interesses = gerar_dados_viagem(origem_usuario, destino_usuario)
        codigo_origem = descobrir_aeroporto(origem_usuario)
        codigo_destino = descobrir_aeroporto(destino_usuario)

        if verificar_voo_disponivel(data_ida, data_volta, codigo_origem, codigo_destino):
            log(f"Datas confirmadas com voo disponível: {data_ida} → {data_volta}")
            break
        else:
            log("Nenhum voo encontrado para as datas geradas, tentando novamente...")
            tentativas += 1

    if tentativas == max_tentativas:
        log("Não foi possível encontrar voo disponível após várias tentativas. Usando datas originais.")

    voos = buscar_voos_reais(codigo_origem, codigo_destino, data_ida, data_volta)
    hoteis = buscar_hoteis_reais(destino_usuario, data_ida, data_volta)
    atracoes = pesquisar_atracoes(destino_usuario, interesses)
    cambio = buscar_cambio(destino_usuario)
    historia = pesquisar_historia(destino_usuario)

    # Retornamos dados intermediários para visualização do prompt
    return {
        "data_ida": data_ida,
        "data_volta": data_volta,
        "interesses": interesses,
        "codigo_origem": codigo_origem,
        "codigo_destino": codigo_destino,
        "voos": voos,
        "hoteis": hoteis,
        "atracoes": atracoes,
        "cambio": cambio,
        "historia": historia
    }

# ==========================================
# 11. FUNÇÃO PARA GERAR E MOSTRAR PROMPT FINAL
# ==========================================
def mostrar_prompt_final(dados_viagem, origem_usuario, destino_usuario):
    prompt = f"""
Gere um roteiro extremamente detalhado para uma viagem de {origem_usuario} para {destino_usuario}.

DADOS REAIS:
✈️ Voos:
{dados_viagem['voos']}

🏨 Hotéis:
{dados_viagem['hoteis']}

🎡 Atrações:
{dados_viagem['atracoes']}

💱 Câmbio:
{dados_viagem['cambio']}

📚 História:
{dados_viagem['historia']}

DATAS:
- Ida: {dados_viagem['data_ida']}
- Volta: {dados_viagem['data_volta']}

Interesses: {dados_viagem['interesses']}

Monte:
- Roteiro dia a dia baseado nos dados reais
- Explicações culturais e históricas
- Custos aproximados
- Justificativa da escolha do roteiro
"""
    log("Prompt final que será enviado ao LLM (truncado se muito longo):")
    print(prompt[:1500] + ("\n... (prompt truncado)" if len(prompt) > 1500 else ""))
    return prompt

# ==========================================
# 12. EXECUÇÃO FINAL
# ==========================================
ORIGEM = "São Paulo"
DESTINO = "Tokyo"

# 1. Gerar dados da viagem
dados_viagem = agente_de_viagens(ORIGEM, DESTINO)

# 2. Mostrar prompt final
prompt_final = mostrar_prompt_final(dados_viagem, ORIGEM, DESTINO)

# 3. Chamar LLM para gerar roteiro
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)
roteiro = llm.invoke(prompt_final).content

# 4. Exibir roteiro em Markdown
print("="*80)
display(Markdown(escapar_markdown(roteiro)))
print("="*80)



🟨 [LOG] Iniciando agente de viagens...


🟨 [LOG] Gerando datas e interesses para a viagem São Paulo → Tokyo via LLM


🟨 [LOG] Datas geradas: 2026-03-28 → 2026-04-06


🟨 [LOG] Interesses: Cultura (templos, museus, pop culture), Gastronomia (sushi, ramen, izakayas), Natureza (cerejeiras em flor, jardins), Vida urbana (Shibuya, Shinjuku), História


🟨 [LOG] Identificando código IATA de São Paulo...


🟨 [LOG] Código encontrado: GRU


🟨 [LOG] Identificando código IATA de Tokyo...


🟨 [LOG] Código encontrado: NRT


🟨 [LOG] Datas confirmadas com voo disponível: 2026-03-28 → 2026-04-06


🟨 [LOG] Buscando voos reais GRU → NRT


🟦 RESULTADO SERPAPI (VOOS) (mostrando até 20 linhas) 

{
    "search_metadata": {
        "id": "69311be06f1e409f8b8a3cfd",
        "status": "Success",
        "json_endpoint": "https://serpapi.com/searches/a662794e2d128261/69311be06f1e409f8b8a3cfd.json",
        "created_at": "2025-12-04 05:28:00 UTC",
        "processed_at": "2025-12-04 05:28:00 UTC",
        "google

Prepare-se para uma aventura inesquecível do Brasil ao Japão! Este roteiro detalhado foi cuidadosamente elaborado para maximizar sua experiência em Tóquio, incorporando seus interesses em cultura, gastronomia, natureza (com foco nas cerejeiras em flor, dado o período da viagem), vida urbana e história, utilizando os dados reais fornecidos.

---

## Roteiro Detalhado: São Paulo a Tóquio (28 Março - 06 Abril 2026)

### Informações Essenciais da Viagem

*   **Período:** 28 de Março de 2026 (Saída de SP) a 06 de Abril de 2026 (Chegada em SP)
*   **Duração em Tóquio:** 7 noites (29 de Março a 05 de Abril)
*   **Interesses:** Cultura (templos, museus, pop culture), Gastronomia (sushi, ramen, izakayas), Natureza (cerejeiras em flor), Vida urbana (Shibuya, Shinjuku), História.

### Escolhas e Justificativas

1.  **Voos:**
    *   **Escolha:** American Airlines (R\\$ 10651, 625 min).
    *   **Justificativa:** É a opção mais econômica entre as fornecidas, com um tempo de voo razoável (aproximadamente 10h 25min). Assumimos que R\\$ 10651 é o valor da passagem de ida e volta por pessoa.
2.  **Hospedagem:**
    *   **Escolha:** Tora Hotel Asakusa (R\\$ 272/noite, Nota: 4.4).
    *   **Justificativa:** Oferece a melhor relação custo-benefício entre os hotéis listados, com uma excelente avaliação (4.4) e um preço acessível. A localização em Asakusa é estratégica para os interesses culturais e históricos, com fácil acesso a templos, mercados tradicionais e estações de metrô, além de ser um bairro charmoso para explorar a pé.
3.  **Período da Viagem e Cerejeiras:**
    *   **Justificativa:** O período de 28 de Março a 06 de Abril é *perfeito* para testemunhar a plena floração das cerejeiras (sakura) em Tóquio, um dos pontos altos da natureza japonesa e um evento cultural de grande significado. O roteiro incluirá parques e locais famosos para *hanami* (contemplação das cerejeiras).

### Cotação do Iene (para referência de custos)

*   **Câmbio:** R\\$ 0,03 = 1 JPY (ou 1 JPY = R\\$ 0,03)
    *   Exemplo: 1.000 JPY = R\\$ 30,00

---

### Custos Aproximados (por pessoa)

*   **Voos (American Airlines):** R\\$ 10.651,00
*   **Hospedagem (Tora Hotel Asakusa):** R\\$ 272/noite * 7 noites = R\\$ 1.904,00
*   **Despesas Diárias (Estimativa):**
    *   **Alimentação:** 6.000 JPY - 9.000 JPY por dia (aprox. R\\$ 180 - R\\$ 270). Inclui café da manhã, almoço, jantar e lanches. Vamos usar uma média de **R\\$ 225/dia**.
    *   **Transporte:** 1.000 JPY - 2.000 JPY por dia (aprox. R\\$ 30 - R\\$ 60). O metrô e trens são eficientes. Vamos usar uma média de **R\\$ 45/dia**.
    *   **Atrações/Entradas:** 1.000 JPY - 3.000 JPY por dia (aprox. R\\$ 30 - R\\$ 90), dependendo das escolhas. Vamos usar uma média de **R\\$ 60/dia**.
    *   **Total Diário Estimado:** R\\$ 225 + R\\$ 45 + R\\$ 60 = **R\\$ 330/dia**
    *   **Total Diário para 8 dias completos em Tóquio:** R\\$ 330 * 8 dias = R\\$ 2.640,00 (considerando 8 dias de gastos em Tóquio, do dia da chegada ao dia anterior à partida).
*   **Total Estimado da Viagem (sem compras/extras):** R\\$ 10.651 (voo) + R\\$ 1.904 (hotel) + R\\$ 2.640 (despesas diárias) = **R\\$ 15.195,00**
*   **Recomendação:** Prever um orçamento extra de R\\$ 3.000 - R\\$ 5.000 para compras, souvenirs e imprevistos.

---

### Roteiro Dia a Dia

#### Dia 1: Sábado, 28 de Março de 2026 - Partida para o Sol Nascente

*   **Manhã/Tarde:** Chegada ao Aeroporto Internacional de Guarulhos (GRU), São Paulo.
*   **Noite:** Embarque no voo da American Airlines para Tóquio. Prepare-se para uma longa, mas emocionante jornada.
*   **Explicação Cultural:** A viagem em si já é uma transição cultural. O fuso horário de Tóquio é 12 horas à frente de São Paulo (UTC+9 vs. UTC-3), o que significa que você "perderá" um dia na chegada, mas o ganhará na volta.
*   **Custo:** Passagem aérea (incluso no total).

#### Dia 2: Domingo, 29 de Março de 2026 - Chegada e Primeiro Contato com Tóquio

*   **Tarde/Noite:** Chegada ao Aeroporto Internacional de Narita (NRT) ou Haneda (HND).
    *   **Procedimento:** Passar pela imigração, pegar as malas.
    *   **Transporte para Asakusa:**
        *   **De Narita (NRT):** Opções como Narita Express (N'EX) para a Estação Tóquio e depois metrô, ou Keisei Skyliner para Ueno e depois metrô para Asakusa. (Tempo: 1h a 1h30min).
        *   **De Haneda (HND):** Keikyu Line para Shinagawa e depois metrô, ou Tokyo Monorail para Hamamatsucho e depois metrô. (Tempo: 45min a 1h).
*   **Hospedagem:** Check-in no **Tora Hotel Asakusa**. Descanse um pouco e se familiarize com o bairro.
*   **Noite:** Jantar leve em um restaurante local em Asakusa. Experimente um **ramen** autêntico ou um **udon** para começar a explorar a gastronomia japonesa. Uma caminhada curta pelos arredores do hotel para sentir a atmosfera noturna do bairro.
*   **Explicação Cultural:** O choque cultural pode ser grande. Observe a organização, a limpeza e o silêncio no transporte público. Asakusa, com suas ruas mais estreitas e edifícios mais baixos, oferece um contraste com a imagem futurista de Tóquio.
*   **Custo Estimado:** Transporte Aeroporto-Hotel (aprox. 3.000 JPY - 4.000 JPY = R\\$ 90 - R\\$ 120), Jantar (aprox. 1.500 JPY - 2.500 JPY = R\\$ 45 - R\\$ 75).

#### Dia 3: Segunda-feira, 30 de Março de 2026 - Asakusa Histórica e Vistas Panorâmicas

*   **Manhã (Cultura/História):**
    *   **Senso-ji Temple:** Comece o dia explorando o templo mais antigo de Tóquio. Caminhe pela **Nakamise-dori**, uma rua comercial vibrante que leva ao templo, cheia de lojas de souvenirs e lanches tradicionais.
    *   **Explicação Histórica:** O Senso-ji foi fundado em 645 d.C. e é dedicado a Kannon, a deusa da misericórdia. Asakusa era o coração da antiga Edo, e o templo é um portal para a história da cidade antes de se tornar Tóquio.
*   **Almoço (Gastronomia):** Experimente um **tempura** ou **soba** em um dos restaurantes tradicionais de Asakusa.
*   **Tarde (Natureza/Vista Urbana):**
    *   **Sumida Park:** Caminhe ao longo do Rio Sumida, que nesta época estará repleto de **cerejeiras em flor**, oferecendo vistas espetaculares do rio e da Tokyo Skytree.
    *   **Tokyo Skytree:** Suba para ter uma vista panorâmica de Tóquio do alto. Escolha o deck Tembo Galleria para a experiência completa.
    *   **Explicação:** A Skytree é um símbolo da Tóquio moderna, contrastando com a Asakusa histórica. Oferece uma perspectiva única da vasta metrópole.
*   **Noite (Gastronomia/Cultura):** Jantar em um **izakaya** (bar japonês com petiscos) em Asakusa, experimentando diversos pratos pequenos e bebidas locais.
*   **Custo Estimado:** Entrada Skytree (aprox. 3.000 JPY = R\\$ 90), Refeições (aprox. 3.000 JPY = R\\$ 90), Transporte local (aprox. 500 JPY = R\\$ 15).

#### Dia 4: Terça-feira, 31 de Março de 2026 - Palácio Imperial, Ginza e Sakura no Coração da Cidade

*   **Manhã (História/Natureza):**
    *   **Jardins do Palácio Imperial (East Garden):** Visite os jardins que circundam o local do antigo Castelo de Edo, residência do Shogun Tokugawa. É um oásis de tranquilidade no centro da cidade.
    *   **Chidorigafuchi Moat:** Caminhe ao longo do fosso do Palácio Imperial, um dos locais mais famosos para ver **cerejeiras em flor**, com a opção de alugar um barco a remo para uma experiência única entre as pétalas.
    *   **Explicação Histórica:** O Palácio Imperial é construído sobre os restos do Castelo de Edo, que foi a sede do poder do xogunato Tokugawa por mais de 250 anos. Em 1868, com a Restauração Meiji, Edo tornou-se Tóquio e a residência imperial foi transferida para cá.
*   **Tarde (Vida Urbana/Cultura):**
    *   **Dieta Nacional de Tóquio (exterior):** Faça uma breve parada para admirar a arquitetura imponente do prédio da Dieta Nacional, o parlamento japonês.
    *   **Ginza:** Explore o bairro mais luxuoso de Tóquio, conhecido por suas lojas de grife, galerias de arte e arquitetura moderna. É um ótimo lugar para observar as pessoas e a elegância japonesa.
*   **Almoço (Gastronomia):** Experimente um almoço mais sofisticado em Ginza ou um kaiten-sushi (sushi de esteira) para uma experiência divertida e acessível.
*   **Noite (Gastronomia):** Jantar em Ginza ou retorne a Asakusa para mais opções locais.
*   **Custo Estimado:** Aluguel de barco (opcional, aprox. 800 JPY = R\\$ 24), Refeições (aprox. 3.500 JPY = R\\$ 105), Transporte local (aprox. 800 JPY = R\\$ 24).

#### Dia 5: Quarta-feira, 1 de Abril de 2026 - Cultura Pop e Natureza em Shibuya, Harajuku e Shinjuku

*   **Manhã (Cultura/Natureza):**
    *   **Meiji Jingu Shrine:** Comece o dia com uma visita a este santuário xintoísta dedicado ao Imperador Meiji e à Imperatriz Shoken. Caminhe pela floresta exuberante que o cerca, um contraste sereno com a agitação da cidade.
    *   **Explicação Cultural:** O xintoísmo é a religião nativa do Japão, focada na adoração de kami (divindades ou espíritos da natureza). Os santuários são locais de purificação e conexão com o sagrado.
*   **Tarde (Vida Urbana/Cultura Pop):**
    *   **Harajuku:** Explore a Takeshita Street, o epicentro da moda jovem e da cultura pop japonesa. Visite as lojas peculiares, cafés temáticos e observe os estilos únicos.
    *   **Shibuya:** Atravesse o famoso Shibuya Crossing, o cruzamento mais movimentado do mundo. Tire uma foto com a estátua de Hachiko, o cão leal.
*   **Noite (Vida Urbana/Vista Urbana):**
    *   **Shinjuku:** Suba ao Tokyo Metropolitan Government Building para uma vista panorâmica gratuita de Tóquio (incluindo o Monte Fuji em dias claros).
    *   **Golden Gai/Kabukicho:** Explore a vida noturna de Shinjuku. Golden Gai é famoso por seus minúsculos bares, enquanto Kabukicho é o distrito da luz vermelha, com letreiros neon vibrantes.
*   **Custo Estimado:** Refeições (aprox. 3.000 JPY = R\\$ 90), Transporte local (aprox. 1.000 JPY = R\\$ 30), Compras (variável).

#### Dia 6: Quinta-feira, 2 de Abril de 2026 - Experiência Gastronômica e Futuro em Odaiba

*   **Manhã (Gastronomia):**
    *   **Tsukiji Outer Market:** Embora o mercado de peixes principal tenha se mudado para Toyosu, o Mercado Exterior de Tsukiji continua sendo um paraíso gastronômico. Experimente frutos do mar frescos, sushi, tamagoyaki (omelete japonesa) e outros petiscos.
    *   **Explicação Cultural:** O mercado de peixes tem uma longa história em Tóquio, sendo um centro vital para a culinária japonesa. A cultura de comer peixe fresco é intrínseca à identidade gastronômica do país.
*   **Tarde (Vida Urbana/Passeio):**
    *   **Passeio de Barco para Odaiba:** Pegue um barco no Rio Sumida (próximo a Asakusa) para Odaiba, uma ilha artificial futurista. O passeio oferece vistas deslumbrantes da Rainbow Bridge e do skyline de Tóquio.
    *   **Odaiba:** Explore as atrações da ilha, como a réplica da Estátua da Liberdade, o Gundam gigante em frente ao DiverCity Tokyo Plaza, e talvez o teamLab Borderless (se estiver aberto ou um museu de arte digital similar).
*   **Noite (Gastronomia):** Jantar em Odaiba, com muitas opções de restaurantes com vista para a baía, ou retorne para Asakusa.
*   **Custo Estimado:** Comida no Tsukiji (aprox. 2.500 JPY = R\\$ 75), Barco para Odaiba (aprox. 1.500 JPY = R\\$ 45), Entradas em Odaiba (se houver, aprox. 2.000 JPY = R\\$ 60), Jantar (aprox. 3.000 JPY = R\\$ 90).

#### Dia 7: Sexta-feira, 3 de Abril de 2026 - Mergulho na Cultura Otaku e Artesanato Japonês

*   **Manhã (Cultura Pop):**
    *   **Akihabara:** Mergulhe no mundo da eletrônica, anime, mangá e games. Visite lojas como Yodobashi Akiba, explore os *arcades* (centros de jogos) e talvez um *maid café* para uma experiência divertida e peculiar.
    *   **Explicação Cultural:** Akihabara é o coração da cultura *otaku* (fã obsessivo) no Japão, refletindo a paixão japonesa por tecnologia e entretenimento.
*   **Tarde (Cultura/Artesanato/Gastronomia):**
    *   **Aula de Sushi Roll:** Participe de uma aula prática para "aprender a fazer sushi roll". É uma experiência gastronômica e cultural muito enriquecedora. (Pesquisar e reservar com antecedência).
    *   *Alternativa:* "Crie seu próprio anel e vivencie o artesanato japonês" (também requer reserva).
*   **Noite (Gastronomia):**
    *   **Ueno:** Dirija-se a Ueno para "provar sashimis, petiscos e ramen". Ueno tem uma variedade de restaurantes e izakayas, muitos deles com uma atmosfera animada.
*   **Custo Estimado:** Compras em Akihabara (variável), Aula de sushi/anel (aprox. 5.000 JPY - 10.000 JPY = R\\$ 150 - R\\$ 300), Jantar em Ueno (aprox. 3.000 JPY = R\\$ 90), Transporte local (aprox. 800 JPY = R\\$ 24).

#### Dia 8: Sábado, 4 de Abril de 2026 - Relaxamento e Descobertas Locais

*   **Manhã (Natureza/Cultura):**
    *   **Shinjuku Gyoen National Garden:** Visite este belíssimo jardim, que combina estilos paisagísticos japonês, francês e inglês. Nesta época, as **cerejeiras em flor** estarão no auge, proporcionando um cenário deslumbrante para um passeio tranquilo.
    *   **Explicação Cultural:** Os jardins japoneses são projetados para evocar a natureza e proporcionar um espaço para a contemplação e a meditação.
*   **Tarde (Cultura Local):**
    *   **Yanaka Ginza Shopping Street:** Explore esta charmosa rua comercial em um bairro mais antigo e tradicional de Tóquio. É um lugar perfeito para sentir a atmosfera de uma Tóquio mais "antiga", com lojas de artesanato, confeitarias e cafés.
*   **Noite (Gastronomia):** Jantar de despedida em um restaurante de sua escolha, talvez revisitando um prato favorito ou experimentando algo novo.
*   **Custo Estimado:** Entrada Shinjuku Gyoen (aprox. 500 JPY = R\\$ 15), Refeições (aprox. 3.500 JPY = R\\$ 105), Transporte local (aprox. 800 JPY = R\\$ 24).

#### Dia 9: Domingo, 5 de Abril de 2026 - Últimos Momentos e Compras de Souvenirs

*   **Manhã (Compras/Cultura):**
    *   **Compras de Última Hora:** Aproveite para comprar os últimos souvenirs em Asakusa (Nakamise-dori) ou em alguma loja de departamento em Ginza ou Shinjuku.
    *   **Museu:** Visite o Tokyo National Museum em Ueno (se não o fez no Dia 3) para uma imersão mais profunda na arte e história japonesas, ou o Edo-Tokyo Museum (verificar se está reaberto/similar) para entender a evolução da cidade.
*   **Almoço (Gastronomia):** Desfrute de um último almoço japonês, talvez um **sushi** de alta qualidade ou um **tonkatsu** (costeleta de porco frita).
*   **Tarde:** Tempo livre para relaxar, organizar as malas ou revisitar um local favorito.
*   **Noite:** Jantar e preparação final para a partida.
*   **Custo Estimado:** Compras (variável), Entrada museu (aprox. 1.000 JPY = R\\$ 30), Refeições (aprox. 3.000 JPY = R\\$ 90), Transporte local (aprox. 500 JPY = R\\$ 15).

#### Dia 10: Segunda-feira, 6 de Abril de 2026 - Retorno a São Paulo

*   **Manhã:** Café da manhã no hotel, check-out.
*   **Transporte para o Aeroporto:** Retorne ao Aeroporto de Narita (NRT) ou Haneda (HND) usando as mesmas opções de transporte da chegada.
*   **Voo:** Embarque no voo da American Airlines de volta para São Paulo.
*   **Chegada em SP:** Devido ao fuso horário, você chegará em São Paulo no mesmo dia 6 de abril, ou na madrugada do dia 7, dependendo do horário do voo e conexões.
*   **Custo Estimado:** Transporte Hotel-Aeroporto (aprox. 3.000 JPY - 4.000 JPY = R\\$ 90 - R\\$ 120).

---

### Justificativa da Escolha do Roteiro

Este roteiro foi montado para proporcionar uma experiência rica e equilibrada em Tóquio, atendendo a todos os interesses especificados:

1.  **Cultura e História:**
    *   **Asakusa (Senso-ji):** Ponto de partida ideal para a história de Edo.
    *   **Palácio Imperial:** Conexão direta com a história dos Shoguns e da Restauração Meiji.
    *   **Meiji Jingu:** Imersão na cultura xintoísta e na história imperial moderna.
    *   **Dieta Nacional:** Representa a história política do Japão moderno.
    *   **Museus:** Opções para aprofundar o conhecimento histórico e artístico.
    *   **Yanaka Ginza:** Preserva a atmosfera de uma Tóquio mais antiga.

2.  **Gastronomia:**
    *   **Ramen, Udon, Tempura, Soba:** Experimentados em Asakusa e Ueno.
    *   **Tsukiji Outer Market:** Foco em frutos do mar frescos e petiscos.
    *   **Aula de Sushi Roll:** Experiência prática e imersiva na culinária.
    *   **Izakayas:** Para provar uma variedade de petiscos e bebidas locais.
    *   **Sashimis e Petiscos em Ueno:** Atende diretamente à sugestão do prompt.

3.  **Natureza (Cerejeiras em Flor):**
    *   O período da viagem é o auge das cerejeiras.
    *   **Sumida Park, Chidorigafuchi Moat, Shinjuku Gyoen National Garden:** Locais estratégicos inclusos para a contemplação das cerejeiras (hanami), garantindo que este interesse seja central na viagem.

4.  **Vida Urbana:**
    *   **Shibuya Crossing, Hachiko:** Ícones da vida urbana moderna.
    *   **Harajuku (Takeshita Street):** Centro da moda jovem e cultura pop.
    *   **Shinjuku (Metropolitan Government Building, Golden Gai/Kabukicho):** Vistas panorâmicas e vida noturna vibrante.
    *   **Ginza:** Elegância e arquitetura moderna.
    *   **Akihabara:** Mergulho na cultura otaku e tecnologia.
    *   **Odaiba:** Visão futurista da cidade.

5.  **Otimização e Logística:**
    *   **Agrupamento Geográfico:** Os dias foram planejados para agrupar atrações próximas, minimizando o tempo de deslocamento e maximizando o tempo de exploração.
    *   **Pacing:** O roteiro oferece um bom equilíbrio entre dias mais intensos e outros mais relaxados, permitindo flexibilidade e tempo para absorver a experiência.
    *   **Hotel em Asakusa:** A escolha do Tora Hotel em Asakusa é estratégica, pois o bairro é um excelente ponto de partida para a cultura e história, além de ter fácil acesso a outras áreas de Tóquio via metrô.

Este roteiro visa proporcionar uma viagem inesquecível, repleta de descobertas culturais, delícias gastronômicas e paisagens deslumbrantes, aproveitando ao máximo o tempo em uma das cidades mais fascinantes do mundo. Tenha uma excelente viagem!